# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Callback Calculations

In [4]:
# number of callbacks for black-sounding names
callbacks_b = sum(data[data.race=='b'].call)

# number of black-sounding names
sound_b = len(data[data.race=='b'])

# number of callbacks for white-sounding names
callbacks_w = sum(data[data.race=='w'].call)

# number of white-sounding names
sound_w = len(data[data.race=='w'])

# callback proportion for black_sounding names
prop_cb_b = callbacks_b/sound_b

# callback proportion for white-sounding names
prop_cb_w = callbacks_w/sound_w

print("The proportion of callbacks for black-sounding names is "+str(round(prop_cb_b,3)))
print("The proportion of callbacks for white-sounding names is "+str(round(prop_cb_w,3)))

The proportion of callbacks for black-sounding names is 0.064
The proportion of callbacks for white-sounding names is 0.097


### Question 1: What test is appropriate for this problem? Does CLT apply?

The appropriate test for this problem will be a **Difference in Proportion Confidence Interval** and **Two-Sided Z-Test**. We are using a **z-statistic** because with the high sample size, the statistic of the sampling distribution will be normally distributed.

##### 1st condition of CLT: Randomization

As stated above, black and white values were **randomly assigned** to the resumes and thus, our samples meet the random samples condition.

##### 2nd condition of CLT: Independence / 10% Rule

In [5]:
"There are" + " " + str(len(data.id)) + " " + "records in the dataset"

'There are 4870 records in the dataset'

With **4870 records** that include both black-sounding and white-sounding names, the 10% rule holds in that the sample  of 4870 records is definitively **less than 10%** of the black-sounding and white-sounding names in the population (billions each).

##### 3rd condition of CLT: Normality

The sample size times the proportion (np) should be **greater than** 10.

In [6]:
# sample size times proportion for black-sounding names
np_b = sound_b* prop_cb_b 
# sample size times proportion for white-sounding names
np_w = sound_w* prop_cb_w

print(np_b, np_w)

157.0 235.0


The normality condition applies as both are **well above 10.**

### Question 2: What are the null and alternate hypotheses?

*Null Hypothesis:* There **is no** difference between the black-sounding and white-sounding sample proportions. 
                    **prop_cb_b - prop_cb_w = 0**

*Alternative Hypothesis:* There **is** a difference between the black-sounding and white-sounding sample proportions.
                    **prop_cb_b - prop_cb_w != 0**
                    
*Significance Level:* 5%

### Question 3: Compute margin of error, confidence interval, and p-value.

In [7]:
diff_sample_prop = prop_cb_w - prop_cb_b # statistic

std_error = math.sqrt((prop_cb_w * (1-prop_cb_w) / sound_w) + (prop_cb_b * (1-prop_cb_b)/sound_b)) # standard erro

z_score = 1.96 # number of standard devations away at a significance level of 95%

In [8]:
moe = z_score*std_error # margin of error

lower = diff_sample_prop - moe # lower level of confidence interval
upper = diff_sample_prop + moe # upper level of confidence interval

print("Margin of Error:", round(moe,3))
print("Confidence Interval:", [round(lower,3),round(upper,3)])

Margin of Error: 0.015
Confidence Interval: [0.017, 0.047]


In [9]:
p = prop_cb_w = prop_cb_b # assuming null hypothesis is true

p_hat = (callbacks_w+callbacks_b) / (sound_w+sound_b) # new proportion assuming null-hypothesis is true

std_error = math.sqrt((2*p_hat*1-p_hat) 
                      /sound_w) # standard error of sampling proportion assuming null-hypothesis is true

z_score = (p - 0)/std_error # calculate z score

p_value = stats.norm.sf(abs(z_score)) #calculate p-value

print('Z-score:', round(z_score,2))
print('P-Value:', round(p_value,2))

Z-score: 11.21
P-Value: 0.0


### Question 4: Write a story describing the statistical significance in the context or the original problem.

**The is a 95% chance that the true difference of white-sounding call back rates and black-sounding call back rates is between .017 and .047**

Thus, we are 95% confident that there is a difference between white-sounding names are more inclined to getting a callback.

The probability of getting a **z-score of 11.21** is **0**  (*p-value = 0.0*), assuming the null-hypothesis is true. Thus, we can assume **statistical significance** in the **difference** between proportions of white/black-sounding names on callback rates.

### Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis **ONLY** looked at the relationship between race (white/black-sounding names) and callback success. There are **numerous other variables** to look and to uncover the correlation between. We will use very similar tests to check for significance in the context of callback success which will be the next step to our analysis.